In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import matplotlib.pyplot as plt
from feature_manager import FeatureManager
from classifier.dnn_classifier import DNNClassifer
from random import randint
from keras import callbacks
from visualizer import *


In [ ]:

data = pd.read_csv("../data/BTCUSDT-1d.csv", parse_dates=["Open Time"], index_col = "Open Time")
fm = FeatureManager()
fm.build_feature(data=data,lags=5)

In [ ]:
classifier = DNNClassifer()
classifier.configure(hl=2, hu = 100, dropout=True, input_dim=len(fm.cols))
classifier.prepare_data(data = fm.df,cols = fm.cols,target_col = "dir", random_state=randint(0,100))
classifier.run()

In [ ]:
acc_list = []
cov_list = []
laps = 5

for i in range (0,laps):
    print("\n======= Lap {} =======".format(i+1))
    callbacks.backend.clear_session()
    classifier = DNNClassifer(neg_cutoff=0.40,pos_cutoff=0.6,epochs=50,train_size=0.7,val_size=0.15)
    classifier.configure(hl=2, hu = 100, dropout=True, input_dim=len(fm.cols))
    classifier.prepare_data(data = fm.df,cols = fm.cols,target_col = "dir", shuffle=True, random_state=i)
    accurracy, coverage = classifier.run()
    classifier.visualize_loss()
    classifier.visualize_accuracy()
    acc_list.append(accurracy)
    cov_list.append(coverage)

acc_arr = np.array(acc_list)
cov_arr = np.array(cov_list)

print("\n======")
print("Accuracy mean: {}, std: {}".format(acc_arr.mean(),acc_arr.std()))
print("Coverage mean: {}, std: {}".format(cov_arr.mean(),cov_arr.std()))
print("======")